In [ ]:
from modules.engine import SolverPool, SolverSelectionStrategy, MySolver, Query
from modules.log_reader import read_qsym_log
import logging

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
class NegationSolverSelectionStrategy(SolverSelectionStrategy):
    def get_solver(self, found_solver: MySolver | None, query: Query) -> MySolver:
        if found_solver is None:
            logging.info("No solver available, Creating a new one")
            solver = self.create_empty_solver()
            solver.upgrade(query.path[:-1], query.constraints[:-1])
            return solver

        solver = found_solver
        if found_solver.stack_path_len < len(query.path) - 1:
            logging.info("Creating a new solver for a longer path")
            solver = self.create_empty_solver()
            solver.copy_from(found_solver)
            solver.upgrade(query.path[solver.stack_path_len:-1],
                           query.constraints[solver.constraint_count:-1])
        else:
            logging.info("Solver is appropriate")
        
        return solver


In [ ]:
queries = read_qsym_log("./sample2_export.log")
# [q.to_json_serializable() for q in queries]
len(queries)

In [ ]:
pool = SolverPool()
pool.selection_strategy = NegationSolverSelectionStrategy()
pool.enable_solve_prefix_assertions()

In [ ]:
logging.getLogger().setLevel(level=logging.DEBUG)

for q in queries:
    pool.solve(q)

logging.getLogger().setLevel(level=logging.INFO)

In [ ]:
print("Solver tree count:", len(pool._solver_trees))
print("Most recently used solver:", list(pool._solvers.items.items())[-1][1]._solver)
print("Statistics:", [solver.statistics for (id, solver) in pool._solvers.items.items()])
s = list(pool._solvers.items.items())[-1][1]._solver
s.to_smt2()